# EPA-Justice QC

This notebook runs an independent validation of the results produced in the `fetch_data_and_export.ipynb` notebook. The goals of the QC are to:


- **Validate the API:** The functions used in the `fetch_data_and_export.ipynb` notebook utilize APIs to find census data, but here we will actually use data from some downloaded CSVs to compare against the results in `data_to_export.csv`. This should prove that the data gathered from the API matches what we see in the raw tabular data.

- **Check the math:** We will do an independent aggregation of the original data to check our math, and keep those original data values visible for inspection.

- **Ensure reasonable values:** A number of QC checks are done to assure that values are within expected ranges, and also to assess missing data.


Note: The 2020 US Census data CSVs (DHC tables P12 and P9, and ACS 5 year tables S1810 and S2701) were downloaded from the [data.census.gov](data.census.gov) online application, after filtering for all counties, places, and tracts in Alaska. The CDC PLACES and SDOH data CSVs for counties, places, and tracts in Alaska were downloaded from the CDC [data portal](https://data.cdc.gov/browse?category=500+Cities+%26+Places).

Note: The downloaded files do not include data from zip code tabulation area (ZCTA) geographies, since they were not used in this project. The codebase does support using ZCTAs, but they are not QC'd here.

## Data Prep

In [830]:
import pandas as pd
import random
import numpy as np
from utilities.luts import *

Load places list and previously generated results.

In [831]:
places = pd.read_csv('tbl/NCRPlaces_Census_04192024.csv')
places.drop(columns=['alt_name', 'region', 'country', 'latitude', 'longitude', 'type'], inplace=True)
results = pd.read_csv('tbl/data_to_export.csv')

Optionally, choose 20 random places for QC, and get corresponding results. This might be useful if you also wanted to do a manual check of the data aggregation, or for further development/troubleshooting of the QC process.

In [832]:
qc_places = places.where(places['id'].isin(random.sample(places['id'].to_list(), 20))).dropna(subset='id')
qc_results = results.where(results['id'].isin(qc_places['id'])).dropna(subset='id')

Or, just check all results.

In [833]:
qc_places = places
qc_results = results

Load Census and CDC data from CSVs. Join them into 2 main files and keep only the columns listed in the lookup tables. There is some table reformatting required here in order to accomplish the merge.

In [834]:
# census data
dhc_p12 = pd.read_csv('qc_data/DECENNIALDHC2020.P12-Data.csv')
dhc_p9 = pd.read_csv('qc_data/DECENNIALDHC2020.P9-Data.csv')
acs5_S1810 = pd.read_csv('qc_data/ACSST5Y2020.S1810-Data.csv')
acs5_S2701 = pd.read_csv('qc_data/ACSST5Y2020.S2701-Data.csv')

# cdc data
places_county = pd.read_csv('qc_data/PLACES__Local_Data_for_Better_Health__County_Data_2023_release_20240506.csv')
places_place = pd.read_csv('qc_data/PLACES__Local_Data_for_Better_Health__Place_Data_2023_release_20240506.csv')
places_tract = pd.read_csv('qc_data/PLACES__Local_Data_for_Better_Health__Census_Tract_Data_2023_release_20240506.csv')

sdoh_county = pd.read_csv('qc_data/SDOH_Measures_for_County__ACS_2017-2021_20240506.csv')
sdoh_place = pd.read_csv('qc_data/SDOH_Measures_for_Place__ACS_2017-2021_20240506.csv')
sdoh_tract = pd.read_csv('qc_data/SDOH_Measures_for_Census_Tract__ACS_2017-2021_20240506.csv')

# standardize columns
for df in [sdoh_county, sdoh_place, sdoh_tract]:
    df.rename(columns={'MeasureID':'MeasureId'}, inplace=True)


In [835]:
# get var lists
census_vars = list(var_dict["dhc"]["vars"].keys()) + (list(var_dict["acs5"]["vars"].keys()))
cdc_vars = list(var_dict["cdc"]["PLACES"]["vars"].keys()) + (list(var_dict["cdc"]["SDOH"]["vars"].keys()))

In [836]:
# merge census data, dropping unused cols
census_data = dhc_p12.merge(
    dhc_p9, how='outer', on=['GEO_ID','NAME']).merge(
        acs5_S1810, how='outer', on=['GEO_ID']).merge(
            acs5_S2701, how='outer', on=['GEO_ID'])
for col in census_data.columns:
    if col not in ['GEO_ID', 'NAME'] and col not in census_vars:
        census_data.drop(columns=col, inplace=True)
# keep multiindex
#census_data.columns = pd.MultiIndex.from_arrays([census_data.columns, census_data.iloc[0].values])
# drop multiindex
census_data = census_data.iloc[1:]

In [837]:
# reformat cdc data, dropping unused cols and keeping only crude prevalence
# pivot and drop unused measures (vars), and merge
cdc_dfs_reformatted = []

cdc_dfs = [places_county, places_place, places_tract, sdoh_county, sdoh_place, sdoh_tract]
cdc_cols = ['LocationID', 'LocationName', 'DataValueTypeID', 'Measure', 'MeasureId', 'Data_Value']

for df in cdc_dfs:
    for col in df.columns:
        if col not in cdc_cols:
            df.drop(columns=col, inplace=True)
    df.drop(df.loc[df['DataValueTypeID'] == 'AgeAdjPrv'].index, inplace=True)
    df.drop(columns='DataValueTypeID', inplace=True)

    df_pivot = df.pivot(index=['LocationID', 'LocationName'], columns=['MeasureId', 'Measure'], values='Data_Value').reset_index()
    df_pivot = df_pivot.droplevel(level=1, axis=1)

    for col in df_pivot.columns:
        if col not in cdc_cols and col not in cdc_vars:
            df_pivot.drop(columns=col, inplace=True)
            
    cdc_dfs_reformatted.append(df_pivot)

cdc_data = pd.concat(cdc_dfs_reformatted[0:3]).merge(pd.concat(cdc_dfs_reformatted[3:]), how='outer', on='LocationID')
cdc_data.columns.name = None

Finally, we have some merged data that will be easier to QC.

In [838]:
census_data.head()

,GEO_ID,P12_001N,P12_002N,P12_003N,P12_004N,P12_005N,P12_006N,P12_020N,P12_021N,P12_022N,...,P9_009N,P9_010N,P9_011N,S1810_C03_001E,S1810_C03_001M,NAME,S2701_C03_001E,S2701_C03_001M,S2701_C05_001E,S2701_C05_001M
1,0500000US02013,3420,2371,38,33,51,38,39,37,53,...,32,11,180,10.7,2.2,"Aleutians East Borough, Alaska",60.3,3.4,39.7,3.4
2,0500000US02016,5232,3432,76,106,87,57,61,45,44,...,238,10,263,7.9,2.7,"Aleutians West Census Area, Alaska",82.8,2.5,17.2,2.5
3,0500000US02020,291247,147894,9746,9879,9964,5657,3232,3852,4935,...,9844,1922,31273,11.2,0.6,"Anchorage Municipality, Alaska",89.4,0.7,10.6,0.7
4,0500000US02050,18666,9747,931,977,881,567,168,209,204,...,2,19,945,11.4,1.0,"Bethel Census Area, Alaska",82.6,2.1,17.4,2.1
5,0500000US02060,844,440,23,37,18,16,13,18,17,...,3,2,130,15.9,4.1,"Bristol Bay Borough, Alaska",83.9,5.1,16.1,5.1


In [839]:
cdc_data.head()

,LocationID,LocationName_x,STROKE,DIABETES,KIDNEY,CHD,CASTHMA,COPD,LocationName_y,REMNRTY,NOHSDP,BROAD,POV150
0,2013,Aleutians East,3.3,13.1,3.0,5.7,7.8,5.4,Aleutians East Borough,87.2,15.3,42.5,22.7
1,2016,Aleutians West,2.5,10.8,2.4,4.5,7.4,4.4,Aleutians West Census Area,77.1,9.0,23.0,11.3
2,2020,Anchorage,2.6,7.9,2.6,4.7,9.4,5.3,Anchorage Municipality,43.9,5.8,7.3,15.1
3,2050,Bethel,4.8,14.8,3.9,7.9,12.8,10.4,Bethel Census Area,90.8,18.0,25.2,43.9
4,2060,Bristol Bay,3.6,10.7,3.4,6.9,10.0,6.9,Bristol Bay Borough,58.8,5.3,23.5,8.0


We need to standardize the GEOIDs in order to merge all the tables together. We will drop the "US" and everything before it for all census-based GEOID columns, and add leading zeros to the CDC location id columns.

In [840]:
qc_places['GEOIDFQ'] = qc_places['GEOIDFQ'].str.split("US").str[1]
census_data['GEO_ID'] = census_data['GEO_ID'].str.split("US").str[1]
cdc_data['LocationID'] = "0" + cdc_data['LocationID'].astype(str)

Now we can merge the tables and replace column names using our short names from the lookup table. Let's view all column names to make sure we have everything we need, then do a last check of the pertinent geography columns to make sure everything looks like it lined up right during the table joining operations.

In [841]:
df = qc_places.merge(census_data, how='left', left_on='GEOIDFQ', right_on='GEO_ID').merge(
    cdc_data, how='left', left_on='GEOIDFQ', right_on='LocationID')

In [842]:
for col in df.columns:
    if col in var_dict["dhc"]["vars"].keys():
        new_col = var_dict["dhc"]["vars"][col]["short_name"]
        df.rename(columns={col : new_col}, inplace=True)
    elif col in var_dict["acs5"]["vars"].keys():
        new_col = var_dict["acs5"]["vars"][col]["short_name"]
        df.rename(columns={col : new_col}, inplace=True)
    elif col in var_dict["cdc"]["PLACES"]["vars"].keys():
        new_col = var_dict["cdc"]["PLACES"]["vars"][col]["short_name"]
        df.rename(columns={col : new_col}, inplace=True)
    elif col in var_dict["cdc"]["SDOH"]["vars"].keys():
        new_col = var_dict["cdc"]["SDOH"]["vars"][col]["short_name"]
        df.rename(columns={col : new_col}, inplace=True)

In [843]:
df.columns

Index(['id', 'name', 'GEOIDFQ', 'PLACENAME', 'AREATYPE', 'COMMENT', 'GEO_ID',
       'total_population', 'total_male', 'm_under_5', 'm_5_to_9', 'm_10_to_14',
       'm_15_to_17', 'm_65_to_66', 'm_67_to_69', 'm_70_to_74', 'm_75_to_79',
       'm_80_to_84', 'm_85_plus', 'total_female', 'f_under_5', 'f_5_to_9',
       'f_10_to_14', 'f_15_to_17', 'f_65_to_66', 'f_67_to_69', 'f_70_to_74',
       'f_75_to_79', 'f_80_to_84', 'f_85_plus', 'total_p9', 'hispanic_latino',
       'white', 'african_american', 'amer_indian_ak_native', 'asian',
       'hawaiian_pacislander', 'other', 'multi', 'pct_w_disability',
       'moe_pct_w_disability', 'NAME', 'pct_insured', 'moe_pct_insured',
       'pct_uninsured', 'moe_pct_uninsured', 'LocationID', 'LocationName_x',
       'pct_stroke', 'pct_diabetes', 'pct_kd', 'pct_hd', 'pct_asthma',
       'pct_copd', 'LocationName_y', 'pct_minority', 'pct_no_hsdiploma',
       'pct_no_bband', 'pct_below_150pov'],
      dtype='object')

In [844]:
geo_cols = ['id', 'name', 'GEOIDFQ', 'PLACENAME', 'AREATYPE', 'GEO_ID', 'NAME', 'LocationID', 'LocationName_x', 'LocationName_y', 'COMMENT']
df[geo_cols]

,id,name,GEOIDFQ,PLACENAME,AREATYPE,GEO_ID,NAME,LocationID,LocationName_x,LocationName_y,COMMENT
0,BORO1,Aleutians East Borough,02013,Aleutians East Borough,County,02013,"Aleutians East Borough, Alaska",02013,Aleutians East,Aleutians East Borough,NaN
1,BORO19,Municipality of Anchorage,02020,Anchorage Municipality,County,02020,"Anchorage Municipality, Alaska",02020,Anchorage,Anchorage Municipality,NaN
2,AK15,Anchorage,02020,Anchorage Municipality,County,02020,"Anchorage Municipality, Alaska",02020,Anchorage,Anchorage Municipality,Data represent information from nearest [COLUM...
3,CENS9,Bethel Census Area,02050,Bethel Census Area,County,02050,"Bethel Census Area, Alaska",02050,Bethel,Bethel Census Area,NaN
4,BORO17,Bristol Bay Borough,02060,Bristol Bay Borough,County,02060,"Bristol Bay Borough, Alaska",02060,Bristol Bay,Bristol Bay Borough,NaN
...,...,...,...,...,...,...,...,...,...,...,...
423,AK430,Willow,0285280,Willow CDP,Census designated place,0285280,"Willow CDP, Alaska",0285280,Willow,Willow,Data represent information from nearest [COLUM...
424,AK556,Willow Creek,0285290,Willow Creek CDP,Census designated place,0285290,"Willow Creek CDP, Alaska",0285290,Willow Creek,Willow Creek,Data represent information from nearest [COLUM...
425,AK431,Wiseman,0285610,Wiseman CDP,Census designated place,0285610,"Wiseman CDP, Alaska",NaN,NaN,NaN,Data represent information from nearest [COLUM...
426,AK432,Womens Bay,0285680,Womens Bay CDP,Census designated place,0285680,"Womens Bay CDP, Alaska",0285680,Womens Bay,Womens Bay,Data represent information from nearest [COLUM...


Finally, convert all values not in geography columns to floats. Some of the CSV NaN values are odd, and do not match the NaNs returned by the API (e.g., we see some `"-"` and `"**"` values in the CSVs). For this reason, we need to use a different approach than is used in `utilities.fuctions.py`.

In [845]:
for col in df.columns:
    if col not in geo_cols:
        df[col] = df[col].apply(lambda x: np.nan if isinstance(x, str) == True and x.replace(".","",1).isdigit() == False else x)
        df[col] = df[col].astype(float)

## Do the math

We will recreate the `compute_dhc()` function, and use it to compute fields in our QC dataframe. 

In [846]:
def compute_dhc(dhc_data):

    dhc_data["m_under_18"] = dhc_data[
        ["m_under_5", "m_5_to_9", "m_10_to_14", "m_15_to_17"]
    ].sum(axis=1, skipna=False)
    dhc_data["f_under_18"] = dhc_data[
        ["f_under_5", "f_5_to_9", "f_10_to_14", "f_15_to_17"]
    ].sum(axis=1, skipna=False)
    dhc_data["total_under_18"] = dhc_data[["m_under_18", "f_under_18"]].sum(
        axis=1, skipna=False
    )
    dhc_data["m_65_plus"] = dhc_data[
        [
            "m_65_to_66",
            "m_67_to_69",
            "m_70_to_74",
            "m_75_to_79",
            "m_80_to_84",
            "m_85_plus",
        ]
    ].sum(axis=1, skipna=False)
    dhc_data["f_65_plus"] = dhc_data[
        [
            "f_65_to_66",
            "f_67_to_69",
            "f_70_to_74",
            "f_75_to_79",
            "f_80_to_84",
            "f_85_plus",
        ]
    ].sum(axis=1, skipna=False)
    dhc_data["total_65_plus"] = dhc_data[["m_65_plus", "f_65_plus"]].sum(
        axis=1, skipna=False
    )
    dhc_data["pct_65_plus"] = round(
        dhc_data["total_65_plus"] / dhc_data["total_population"] * 100, 2
    )  
    dhc_data["pct_under_18"] = round(
        dhc_data["total_under_18"] / dhc_data["total_population"] * 100, 2
    )  
    dhc_data["pct_hispanic_latino"] = round(
        dhc_data["hispanic_latino"] / dhc_data["total_p9"] * 100, 2
    )
    dhc_data["pct_white"] = round(dhc_data["white"] / dhc_data["total_p9"] * 100, 2)
    dhc_data["pct_african_american"] = round(
        dhc_data["african_american"] / dhc_data["total_p9"] * 100, 2
    )
    dhc_data["pct_amer_indian_ak_native"] = round(
        dhc_data["amer_indian_ak_native"] / dhc_data["total_p9"] * 100, 2
    )
    dhc_data["pct_asian"] = round(dhc_data["asian"] / dhc_data["total_p9"] * 100, 2)
    dhc_data["pct_hawaiian_pacislander"] = round(
        dhc_data["hawaiian_pacislander"] / dhc_data["total_p9"] * 100, 2
    )
    dhc_data["pct_other"] = round(dhc_data["other"] / dhc_data["total_p9"] * 100, 2)
    dhc_data["pct_multi"] = round(dhc_data["multi"] / dhc_data["total_p9"] * 100, 2)

    return dhc_data

In [847]:
df = compute_dhc(df)

## Compare

Now for each field, we can compare the values in our original results table with the independent QC results we just created. If nothing is printed, there are no discrepancies between the results generated thru API calls and the QC results generated by the downloaded CSVs.

In [848]:
# list columns that need to be QC'd
qc_cols = []
for col in results.columns:
    if col not in geo_cols + ['areatype', 'placename', 'comment', 'GEOID']:
        qc_cols.append(col)

#iterate thru qc dataframe, and call rows from results dataframe to compare
for index_, row in df.iterrows():
    # exclude Eagle River and JBER from the QC.... the aggregations for these places were already reviewed in detail
    # see fetch_data_and_export.ipynb for that QC process
    if row['id'] not in ['AK103', 'AK439']:
        result_df = results[results['id'] == row['id']]
        for col in qc_cols:
            if np.isnan(row[col]) and np.isnan(result_df[col].values[0]):
                pass
            elif row[col] != result_df[col].values[0]:
                print(f"Error for {row['id']}: {row['name']} ({row['GEOIDFQ']}) in column {col}:")
                print(f"Result is {result_df[col].values[0]}, but QC value is {row[col]}")